# Быстрый старт

В этом разделе вы узнаете как можно извлекать структурированную информацию с помощью GigaChat и вызова инструментов/функций.

## Подготовка

Для получения [структурированных данных](/docs/modules/model_io/chat/structured_output) примере используются [модели GigaChat с поддержкой функций](https://developers.sber.ru/docs/ru/gigachat/models#modeli-dlya-generatsii).

In [ ]:
!pip install gigachain_community

## Схема данных

Сначала нужно описать какую информацию требуется извлечь из текста.

Для описания примера схемы персональных данных используется Pydantic.

In [16]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Информация о человеке."""

    # Док-строка выше, передается в описании функции
    # и помогает улучшить результаты работы LLM

    # Обратите внимание:
    # * Все поля — необязательные (`optional`). Это позволяет модели не извлекать неописанные поля.
    # * У каждого поля есть описание (`description`), которое передается в модель, в описании аргументов функции.
    # Хорошее пописание помогает повысить качество извлечения.
    name: Optional[str] = Field(..., description="Имя человека")
    hair_color: Optional[str] = Field(
        ..., description="Цвет волос человека, заполни если известен"
    )
    height_in_meters: Optional[float] = Field(
        ..., description="Высота человека в метрах."
    )

При определении схем придерживайтесь следующих правил:

* Описывайте атрибуты и саму схему. Описания передаются в модель и используются для повышения качества извлечения информации.
* Не вынуждайте модель придумывать данные. В примере выше атрибуты отмечены как необязательные (`Optional`), что позволяет модели возвращать `None`, если она не знает ответа.

## Создание экстрактора

Пример ниже демонстрирует код экстрактора информации на основе заданной схемы.

In [17]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

# Определяем промпт: добавляем инструкции и дополнительный контекст
# На этом этапе можно:
# * Добавить примеры работы функций, для улучшения качества извлечения информации
# * Предоставить дополнительную информацию о том какие данные и откуда будут извлекаться
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ты эксперт в извлечении информации из текста. "
            "Извлекай только релевантную информацию из текста. "
            "Если ты не знаешь значение аттрибута, "
            "который нужно извлечь, поставь аттрибуту null.",
        ),
        # О том как повысить качество работы с помощью примеров
        # читайте в рекомендациях
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

Для корректной работы потребуется модель GigaChat с поддержкой вызова инструметов/функций.

In [18]:
from langchain_community.chat_models.gigachat import GigaChat

llm = GigaChat(
    timeout=6000,
    model="GigaChat-Pro",
    temperature=0.01,
)

runnable = prompt | llm.with_structured_output(schema=Person)

Вызов цепочки.

In [19]:
text = "Алан Смит блондин, 1.85 метра высотой"
runnable.invoke({"text": text})

Giga generation stopped with reason: function_call


Person(name='Алан Смит', hair_color='блондин', height_in_meters=1.85)

## Извлечение нескольких сущностей

В большинстве задача вам понадобится извлекать из текста не одну сущсноть, а список.

Для этого вы можете создавать вложенные модели с помощью pydantic.

In [20]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Информация о человеке."""

    # Док-строка выше, передается в описании функции
    # и помогает улучшить результаты работы LLM

    # Обратите внимание:
    # * Все поля — необязательные (`optional`). Это позволяет модели не извлекать неописанные поля.
    # * У каждого поля есть описание (`description`), которое передается в модель, в описании аргументов функции.
    # Хорошее пописание помогает повысить качество извлечения.
    name: Optional[str] = Field(..., description="Имя человека")
    hair_color: Optional[str] = Field(
        ..., description="Цвет волос человека, заполни если известен"
    )
    height_in_meters: Optional[float] = Field(
        ..., description="Высота человека в метрах."
    )


class Data(BaseModel):
    """Информация о людях."""

    # Создание модели, с помощью которой можно извлекать информацию о нескольких людях
    people: List[Person]

:::important

В этом разделе представлен общий пример извлечения информации, который может демонстрировать не самое высокое качество.

В разделе Руководства вы найдете информацию о том как повысить качество извлечения с помощью образцовых примеров.

:::

In [22]:
runnable = prompt | llm.with_structured_output(schema=Data)
text = (
    "Мое имя Джо, мои волосы черные и я 1.75 метра высотой. "
    "У Анны такие же волосы как у меня и она на 10 сантиметров меньше меня."
)
runnable.invoke({"text": text})

Giga generation stopped with reason: function_call


Data(people=[Person(name='Джо', hair_color='черный', height_in_meters=1.75), Person(name='Анна', hair_color='черный', height_in_meters=1.65)])

:::note

Когда схема подразумевает извлечение нескольких сущностей, она также позволяет модели не извлекать никакие сущности и возвращать пустой список, если в тексте нет подходящих данных.

Как правило это полезно, так как позволяет явно задать обязательные атрибуты сущности без необходимости вынуждать модель обнаруживать такую сущность.

:::

## Смотрите также

* Используйте [образцы кода](/docs/use_cases/extraction/how_to/examples) для более эффективного извлечения информации.
* Узнайте [что делать](/docs/use_cases/extraction/how_to/handle_long_text), когда размер текста превышает объем контекста модели.
* Изучите как использовать загрузчики документов и парсеры GigaChain для [извлечения данных из файлов](/docs/use_cases/extraction/how_to/handle_files) вроде PDF.
* Ознакомьтесь с [примером получения структурированной информации](/docs/use_cases/extraction/how_to/parse) от моделей, которые не поддерживают работу с инструментами или функциями. 